In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 1, 0])              # Boundary condition function coefficients
dtype = torch.float                  # Tensor data type
supervised_samples = {n for n in range(16)}
unsupervised_samples = {n for n in range(16, 16)}

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Specify right hand side and stiffness matrix
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
K = PoissonFEM.StiffnessMatrix(mesh)
rhs.set_rhs_stencil(mesh, K)

In [5]:
trainingData = dta.StokesData(supervised_samples, unsupervised_samples)
trainingData.read_data()
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, trainingData.output_resolution**2)

In [7]:
model = gs.GenerativeSurrogate(rom, trainingData)
# model = gs.GenerativeSurrogate()
# model.load()

In [8]:
# optimize lambdac first
lambdac_iterations = 3e4
for k in range(model.data.n_supervised_samples):
    print('sample = ', k)
    lr_init = 1e-2
    model.lambdacOpt.param_groups[0]['lr'] = lr_init
    lambdac_iter = 0
    while lambdac_iter < lambdac_iterations and model.lambdacOpt.param_groups[0]['lr'] > 1e-4 * lr_init:
        model.lambdac_step(k, lambdac_iter)
        lambdac_iter += 1

sample =  0
loss_lambda_c =  45945471369216.0
loss_lambda_c =  448313819136.0
loss_lambda_c =  378070237184.0
loss_lambda_c =  361510764544.0
loss_lambda_c =  351929892864.0
loss_lambda_c =  345759711232.0
loss_lambda_c =  341650309120.0
loss_lambda_c =  338662195200.0
loss_lambda_c =  336362274816.0
loss_lambda_c =  334578188288.0
loss_lambda_c =  333210025984.0
loss_lambda_c =  332175310848.0
Epoch  4512: reducing learning rate of group 0 to 1.0000e-03.
Epoch  4529: reducing learning rate of group 0 to 1.0000e-04.
Epoch  4545: reducing learning rate of group 0 to 1.0000e-05.
Epoch  4561: reducing learning rate of group 0 to 1.0000e-06.
sample =  1
loss_lambda_c =  55534564671488.0
Epoch  4577: reducing learning rate of group 0 to 1.0000e-03.
Epoch  4593: reducing learning rate of group 0 to 1.0000e-04.
Epoch  4609: reducing learning rate of group 0 to 1.0000e-05.
Epoch  4625: reducing learning rate of group 0 to 1.0000e-06.
sample =  2
loss_lambda_c =  821097486352384.0
Epoch  4641: 

In [9]:
model.fit(n_steps=50, with_precisions=False, z_iterations=200, thetac_iterations=10000, lambdac_iterations=50)

step =  0
loss z =  736561.625
loss z =  736539.5
loss z =  736499.25
loss z =  736430.9375
loss z =  736333.1875
loss z =  736203.0625
loss z =  736035.25
loss z =  735823.5625
loss z =  735562.25
loss z =  735245.4375
loss z =  734869.8125
loss z =  734435.9375
loss z =  733950.3125
loss z =  733424.5625
loss z =  732875.375
loss z =  732322.125
loss z =  731784.625
loss z =  731278.5625
loss z =  730820.25
loss z =  730061.25
loss z =  729761.25
loss z =  729510.0625
loss z =  729303.5
loss z =  729135.5625
loss z =  728999.875
loss z =  728891.9375
loss z =  728804.6875
loss z =  728734.625
loss z =  728677.0625
loss z =  728631.3125
loss z =  728596.0
loss z =  728566.0625
loss z =  728537.1875
loss z =  728515.75
loss z =  728494.0625
loss z =  728478.3125
loss z =  728470.875
loss z =  728456.875
loss z =  728439.75
loss z =  728431.5
loss z =  728424.0
loss z =  728422.1875
loss z =  728421.875
loss z =  728419.0625
loss z =  728418.5
loss z =  728416.8125
loss z =  728419.3125

loss_c =  471.18133544921875
loss_c =  463.81695556640625
loss_c =  456.4369201660156
loss_c =  449.02142333984375
loss_c =  441.5534362792969
loss_c =  434.01849365234375
loss_c =  426.4046325683594
loss_c =  418.7023620605469
loss_c =  410.9044189453125
loss_c =  403.005859375
loss_c =  395.0033874511719
loss_c =  386.8956604003906
loss_c =  378.68280029296875
loss_c =  370.366455078125
loss_c =  361.9493103027344
loss_c =  353.4353942871094
loss_c =  344.82952880859375
loss_c =  336.13775634765625
loss_c =  327.36669921875
loss_c =  318.52410888671875
loss_c =  309.6183776855469
loss_c =  300.6587829589844
loss_c =  291.65533447265625
loss_c =  282.618896484375
loss_c =  273.5609436035156
loss_c =  264.493896484375
loss_c =  255.43081665039062
loss_c =  246.38552856445312
thetac step =  73.9135468006134 s
step =  1
loss z =  33629.265625
loss z =  42514.109375
loss z =  21288.640625
loss z =  13345.576171875
loss z =  9678.931640625
loss z =  7407.8037109375
loss z =  6060.775390625

Epoch  6610: reducing learning rate of group 0 to 1.0000e-05.
sample ==  6
loss_lambda_c =  487681154875392.0
Epoch  6626: reducing learning rate of group 0 to 1.0000e-03.
Epoch  6642: reducing learning rate of group 0 to 1.0000e-04.
Epoch  6658: reducing learning rate of group 0 to 1.0000e-05.
sample ==  7
loss_lambda_c =  251274276634624.0
Epoch  6674: reducing learning rate of group 0 to 1.0000e-03.
Epoch  6690: reducing learning rate of group 0 to 1.0000e-04.
Epoch  6706: reducing learning rate of group 0 to 1.0000e-05.
sample ==  8
loss_lambda_c =  65510037258240.0
Epoch  6722: reducing learning rate of group 0 to 1.0000e-03.
Epoch  6738: reducing learning rate of group 0 to 1.0000e-04.
Epoch  6754: reducing learning rate of group 0 to 1.0000e-05.
Epoch  6770: reducing learning rate of group 0 to 1.0000e-06.
sample ==  9
loss_lambda_c =  215556657512448.0
Epoch  6786: reducing learning rate of group 0 to 1.0000e-03.
Epoch  6802: reducing learning rate of group 0 to 1.0000e-04.
Epo

sample ==  14
loss_lambda_c =  425796514611200.0
Epoch  7825: reducing learning rate of group 0 to 1.0000e-03.
Epoch  7841: reducing learning rate of group 0 to 1.0000e-04.
Epoch  7857: reducing learning rate of group 0 to 1.0000e-05.
sample ==  15
loss_lambda_c =  19472500916224.0
Epoch  7873: reducing learning rate of group 0 to 1.0000e-03.
Epoch  7889: reducing learning rate of group 0 to 1.0000e-04.
Epoch  7905: reducing learning rate of group 0 to 1.0000e-05.
lambdac step =  1.3009321689605713 s
loss_c =  176.83328247070312
Epoch 10078: reducing learning rate of group 0 to 2.0000e-02.
Epoch 10089: reducing learning rate of group 0 to 4.0000e-03.
Epoch 10100: reducing learning rate of group 0 to 8.0000e-04.
Epoch 10111: reducing learning rate of group 0 to 1.6000e-04.
Epoch 10122: reducing learning rate of group 0 to 3.2000e-05.
Epoch 10133: reducing learning rate of group 0 to 6.4000e-06.
thetac step =  0.4677610397338867 s
step =  3
loss z =  6597.0693359375
loss z =  10137.63281

loss_f =  16.134037017822266
loss_f =  15.865945816040039
loss_f =  15.608524322509766
loss_f =  15.361631393432617
loss_f =  15.124823570251465
loss_f =  14.897549629211426
thetaf step =  94.93670725822449 s
sample ==  0
sample ==  1
sample ==  2
sample ==  3
sample ==  4
sample ==  5
sample ==  6
sample ==  7
sample ==  8
sample ==  9
sample ==  10
sample ==  11
sample ==  12
sample ==  13
sample ==  14
sample ==  15
lambdac step =  0.0032701492309570312 s
loss_c =  165.431396484375
Epoch 10210: reducing learning rate of group 0 to 1.0000e-03.
Epoch 10221: reducing learning rate of group 0 to 2.0000e-04.
Epoch 10232: reducing learning rate of group 0 to 4.0000e-05.
Epoch 10243: reducing learning rate of group 0 to 8.0000e-06.
Epoch 10254: reducing learning rate of group 0 to 1.6000e-06.
Epoch 10265: reducing learning rate of group 0 to 3.2000e-07.
thetac step =  0.4186978340148926 s
step =  5
loss z =  664668.8125
loss z =  3426.40966796875
loss z =  1334.35986328125
loss z =  1170.2

KeyboardInterrupt: 

In [ ]:
model.fit(n_steps=50, z_iterations=100, with_precisions=True)

In [10]:
model.log_lambdac_mean

tensor([[-10.7111, -10.6361, -11.9003, -12.0129, -12.5206, -10.2839, -11.5509,
         -12.7573, -11.3595, -10.9674, -12.2795, -10.3174, -11.9982, -11.1459,
         -11.2162, -11.3010],
        [ -4.7459,  -3.7773,  -4.8892,  -4.1067,  -3.8800,  -3.8114,  -4.7494,
          -3.7266,  -5.0267,  -3.5613,  -4.6837,  -3.5709,  -5.5742,  -3.4835,
          -5.0206,  -3.2741],
        [ -3.9399,  -3.8375,  -3.3843,  -3.8173,  -3.0564,  -4.4577,  -3.8999,
          -4.2297,  -4.1843,  -3.1487,  -3.3023,  -3.4083,  -4.6911,  -3.9021,
          -3.5473,  -3.5670],
        [ -3.8914,  -2.9802,  -4.3098,  -3.7129,  -3.7588,  -2.3178,  -3.4761,
          -3.0459,  -3.5163,  -3.0546,  -3.5428,  -3.7573,  -4.2452,  -3.1094,
          -2.7218,  -2.9236],
        [ -2.6843,  -3.2286,  -1.6321,  -3.1479,  -3.5123,  -2.1555,  -4.2139,
          -3.7112,  -2.6154,  -2.8926,  -2.6891,  -2.4665,  -2.7087,  -2.6506,
          -2.2848,  -3.0849],
        [ -1.9935,  -3.7032,  -2.7261,  -2.3773,  -2.4934,  

In [ ]:
# model = gs.GenerativeSurrogate()
# model.load()

In [11]:
test_samples = {n for n in range(0, 4)}
testData = dta.StokesData(unsupervised_samples=test_samples)
testData.read_data()
# trainingData.plotMicrostruct(1)
testData.reshape_microstructure_image()

In [ ]:
uf_pred, Z = model.predict(testData, max_iterations=1000)

loss_z_prediction =  293811.90625
loss_z_prediction =  18.48788833618164


In [ ]:
uf_pred[0]

In [ ]:
model.fit(n_steps=10)

In [ ]:
x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose

In [ ]:
fig = plt.figure()

In [ ]:
ax = plt.axes(projection='3d')

In [ ]:
ax.plot_surface(x, y, np.reshape(uf_pred[0], (129, 129)),cmap='viridis', edgecolor='none')

In [ ]:
pp = np.reshape(model.data.P[0, :], (129, 129))

In [ ]:
ax.plot_surface(x, y, pp.detach().numpy())

In [ ]:
model.plot_input_reconstruction()

In [ ]:
model.plot_generated_microstructures()